# Helium Ground state by DFT

This notebook calculates the Helium atom ground state using DFT. The notebook is based on chapter 5 of Thijseen's Computational Physics.

In [218]:
import numpy as np

In [219]:
#constants
r_max = 10
h = 1e-3
secMin = 1e-3
EAcc = 1e-3

rs = np.arange(0,r_max,step=h)
rsF = np.flip(rs)
N = rs.shape[0]

In [220]:
def getRadialWavefunc(E,VHs,Vxs):
    """
    Finds the radial wave equation associated with energy E and hartree potential VHs. Uses Verlet method.
    """
    us = np.empty(N)
    VHsF = np.flip(VHs)
    VxsF = np.flip(Vxs)

    us[0] = rsF[0]*np.exp(-rsF[0])
    us[1] = rsF[1]*np.exp(-rsF[1])

    for i in range(2,N):
        us[i] = 2*us[i-1] - us[i-2] + 2*(VHsF[i-1] + VxsF[i-1] - 2/rsF[i-1] - E)*us[i-1]*h**2
    
    us /= np.sqrt(h*np.sum(us**2))

    return np.flip(us)

In [221]:
def secant(x0,x1,f):
    """
    Applies secant method.
    """
    x2 = (x1*f(x0) - x0*f(x1))/(f(x0) - f(x1))
    if abs(f(x2)) < secMin:
        return x2
    else:
        return secant(x0,x2,f)

In [222]:
def getVHs(us):
    """
    Finds V_H for a given radial solution. Uses Verlet method.
    """
    Us = np.empty(rs.shape)

    Us[0] = 0
    Us[1] = h

    for i in range(2,rs.shape[0]):
        Us[i] = 2*Us[i-1] - Us[i-2] - (us[i-1]**2/rs[i-1])*h**2 

    #applies correct boundary conditions
    alpha = (h*np.sum(us**2) -  Us[-1])/rs[-1]
    Us += alpha*rs

    VHs = Us
    VHs[1:] /= rs[1:] 
    return 2*VHs


In [223]:
def getVxS(us):
    """
    Function that finds the local density exchange potential.
    """
    Vxs = -(3*us**2/(2*np.pi**2))**(1/3)
    Vxs[1:] /= rs[1:]**(2/3)
    return Vxs

In [224]:
def findGroundState(us,EPrev):
    """
    Recursively solves DFT Helium problem to deduce the ground state.
    """
    VHs = getVHs(us)
    Vxs = getVxS(us)

    #finds us with correct boundary conditions
    getU0 = lambda E: getRadialWavefunc(E,VHs,Vxs)[0]
    epsilon = secant(-3,-1,getU0)

    usNew = getRadialWavefunc(epsilon,VHs,Vxs)
    VHsNew = getVHs(usNew)
    VxsNew = getVxS(usNew)
    ENew = 2*epsilon - h*np.sum(VHsNew*usNew**2) - 0.5*h*np.sum(VxsNew*usNew**2)

    if abs(ENew - EPrev) < EAcc:
        return ENew
    return findGroundState(usNew,ENew)


In [225]:
#simple guess of radial solution
us0 = getRadialWavefunc(-2,np.zeros(N),np.zeros(N))

#finds radial solution
print(f"Energy found: {np.round(findGroundState(us0,-2),3)} Hartree. Experimental value is: -2.901 Hartree")


Energy found: -2.725 Hartree. Experimental value is: -2.901 Hartree
